<a href="https://colab.research.google.com/github/gitudaniel/fastai-course-notes/blob/main/Starting_MNIST_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai --upgrade

     |████████████████████████████████| 194kB 7.5MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 239kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:


In [ ]:
pip install fastbook

In [ ]:
# import fastbook
# fastbook.setup_book()

In [ ]:
from fastbook import *

In [ ]:
import fastai
print(fastai.__version__)

In [ ]:
from fastai.vision.all import *
#from utils import *

matplotlib.rc('image', cmap='Greys')

- We're going to start simple and gradually scale up.

- Fastai has a simpler version of MNIST with only 3's and 7's

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

- `path.ls()` prints a count of items and a list of files relative to where our MNIST_SAMPLE was untarred.

- `path` is a `pathlib` path object. 

- `Pathlib` is part of the standard library but `ls` is not an inbuild method but a method added by the creators of fastai to make it easier.

In [ ]:
# show what ls is
path.ls?

In [ ]:
# Show what ls does
path.ls??

In [ ]:
# show the documentation
doc(path.ls)

In [ ]:
(path/'train').ls()

- We see that the data in our train folder is labelled as "3" or "7"

In [ ]:
# sorted() ensures we get the same order of files
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

threes

In [ ]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

In [ ]:
type(im3)

- PIL is the Python Imaging library; the most popular library for working with images on Python by far.

- Our image is a Png file

- To view the numbers that make up this image, we have to convert it into a *Numpy array* or a *PyTorch tensor*.

- A tensor is a PyTorch version of a Numpy array.

- A PyTorch tensor and a Numpy array behave nearly identically much if not most of the time.

- The key difference is that a PyTorch tensor can also be computed on a GPU and not just a CPU.

In [ ]:
# Numbers from the top-left of the image as a Numpy array
array(im3)[4:10,4:10]

In [ ]:
# Numbers from the top-left of the image as a PyTorch tensor
tensor(im3)[4:10,4:10]

- We can slice the array to pick just the part with the top part of the digit in it, and use a Pandas DataFrame to color-code the values using a gradient.

- This shows us clearly how the image is created from the pixel values.

In [ ]:
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:15,4:22])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

- The background white pixels are stored as the number zero, black is the number 255 and shades of grey are between the two.

- The entire image contains 28 pixels across and 28 pixels down, for a total of 768 pixels.

# Pixel similarity

- As a first try, let's find the average pixel value for every pixel of the threes and do the same for each of the sevens.

- This will give us three group averages which will serve as an "ideal" 3 and 7.

- For classification, we see which of these two "ideal" digits the image is most similar to.

- This will make a good baseline.

- `Baseline` - a super simple model that should be very easy to program from scratch with very little magic and will be better than a random model.

- Always start with a reasonable baseline and build on top of it.

- Step one of our model is to get the average pixel values for each of out two groups.

In [ ]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors), len(seven_tensors)

- We check that one of the images looks okay.

- Remember that our images are now tensors. Jupyter displays tensors as values since it only displays images by default for PIL objects.

- To display the tensor as an image we use fastai's `show_image` function

In [ ]:
show_image(three_tensors[1])

In [ ]:
type(three_tensors)

In [ ]:
three_tensors[1].shape

- We need to get the average of all of those threes.

- Currently `three_tensors` is a list of 28 x 28 images.

- We want to stack all those images into a single three-dimensional tensor to make it easier to perform mathematical computations on it.

- PyTorch comes with a function called `stack`.

- Some operations in PyTorch, such as taking a mean require us to cast our integer types to float types.

- The pixels in an image range from 0 - 255

- In computer vision, when working with floats, they are expected to be between zero and one, so we'll divide by 255

In [ ]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape

- `Rank` - number of axes (dimensions) in a tensor.

- `Shape` - size of each axis of a tensor.

- `stacked_threes` is now a rank 3 tensor because it has 3 axes.

- We get the rank of a tensor by getting the length of its shape or getting the number of dimensions.

- We now have 6131 images each of size 28 x 28 pixels.

- The semantics of a tensor are entirely up to us and how we construct it. The first axis doesn't have to be the number of images

In [ ]:
# length of its shape
len(stacked_threes.shape)

In [ ]:
# number of dimensions
stacked_threes.ndim

- We can now compute what the ideal three and seven look like.

- For every pixel position, we calculate the mean over all images by taking the mean along dimension zero of our stacked, rank-3 tensor.

- Dimension zero is the dimension which indexes over all the images.

In [ ]:
mean3 = stacked_threes.mean(0)
show_image(mean3)

In [ ]:
mean3.shape

`stacked_threes.mean()` returns the average pixel across the rank-3 tensor

In [ ]:
stacked_threes.mean()

In [ ]:
mean7 = stacked_sevens.mean(0)
show_image(mean7)

- We can now pick an arbitrary "3" and measure its distance from each of these ideal digits.

In [ ]:
a_3 = stacked_threes[1]
show_image(a_3)

In [ ]:
a_7 = stacked_sevens[1]
show_image(a_7)

- We can't just add up the differences between the pixels of this digit and the ideal digit.

- This is because some differences will be positive and others will be negative and these differences cancel out and shows that the image has zero total differences from the ideal.

- This is very misleading.

- There's two ways data scientists measure distance in this context:
  - `L1 norm/Mean Absolute Difference` - Take the mean of the absolute value of differences (replace negatives with positives and get the mean)
  - `L2 norm/Root Mean Squared Error (RMSE)` - Take the mean of the square of differences and then take the square root.

- Intuitively the difference between `L1 norm` and `mean squared error (MSE)` is that the latter will penalize bigger mistakes more heavily than the former and be more lenient with small mistakes.

In [ ]:
dist_3_abs = (a_3 - mean3).abs().mean()
dist_3_sqr = ((a_3 - mean3)**2).mean().sqrt()
dist_3_abs, dist_3_sqr

In [ ]:
dist_7_abs = (a_3 - mean7).abs().mean()
dist_7_sqr = ((a_3 - mean7)**2).mean().sqrt()
dist_7_abs, dist_7_sqr

- The distance of our digit to the mean of three is closer than its distance to the mean of seven. This is true for both the `mean absolute error` and the `root mean squared error`.

- This means that we can predict that our digit is a three.

- In this case our simple model gives a correct prediction.

- PyTorch already provides both the `MAE` & `RMSE` as loss functions inside the `torch.nn.functional` (imported as `F`).

- In it *MSE* refers to `mean squared error` and *L1* refers to the `mean absolute error` (`L1 norm` in math)

In [ ]:
F.l1_loss(a_3.float(), mean7), F.mse_loss(a_3.float(), mean7).sqrt()

# Computing metrics using broadcasting

- A metric tells us how good our model is by comparing the prediction to the correct label.

- We use *accuracy* as a metric in classification models since it is understandable to most people

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255

valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

valid_3_tens.shape, valid_7_tens.shape

- We want to write a function `is_3` that will decide if an image is a 3 or a 7 depending on which it is closer to.

- For this, we'll need to define a notion of distance, i.e. a function that calculates the distance between two images.

- We'll do this using the mean absolute error

In [ ]:
# mean((-1, -2)) -> take the mean along the last and the second last axis i.e the x and y axis
def mnist_distance(a, b): return (a - b).abs().mean((-1, -2))
mnist_distance(a_3, mean3)

- In order to calculate overall accuracy, we'll need to calculate the distance to the ideal three for every image in the validation set.

- Our distance function is designed for comparing two single images but can work for our stacked tensor through `broadcasting`.

- `Broadcasting` -> a PyTorch feature that automatically expands the tensor of the smaller rank to have the same size as the one with the larger rank.

- In our case it treats `mean3` as 1010 copies of the same image and subtracts each of those copies from each "three" in `valid_3_tens

In [ ]:
valid_3_distance = mnist_distance(valid_3_tens, mean3)
valid_3_distance, valid_3_distance.shape

In [ ]:
def is_3(x): return mnist_distance(x, mean3) < mnist_distance(x, mean7)

In [ ]:
is_3(a_3), is_3(a_3).float()

- Thanks to `broadcasting`, we can test it on the full validation set of threes.

- This enables us to get rid of loops since loops make things slower moreso on the GPU.

In [ ]:
is_3(valid_3_tens)

In [ ]:
accuracy_3s = is_3(valid_3_tens).float().mean()

accuracy_7s = (1 - is_3(valid_7_tens).float()).mean()

accuracy_3s, accuracy_7s, (accuracy_3s + accuracy_7s)/2

- As we can see, we have a model with a 95% accuracy in predicting 3s and 7s using broadcasting.

- Our model however, cannot be defined as a machine learning model according to Arthur Samuel i.e *Automatic means of testing the actual effectiveness of any parameter assignment in terms of actual performance and provide a mechanism for altering the weight assignment so as to maximise the performance.*

- To do better, let's try a system that does some real learning, i.e. can automatically modify itself to improve its performance.

# Stochastic Gradient Descent (SGD)

- Instead of trying to find the similarity of an image with an "ideal" image, we could instead look at each individual pixel, and come up with a set of weights for each pixel, such that the highest weights are associated with those pixels most likely to be black for a particular category.

- For instance, pixels towards the bottom right are not very likely to be activated for a seven so they should have low weight for a seven, but are more likely to be activated for an eight so they should have high weight for an eight.

- We can represent this as a function:
    ```python
      def pr_eight(x,w) = (x*w).sum()
    ```

- Here we are assuming that `X` is an image represented as a vector and the weights are a vector `W`.

- If the image we're looking at has pixels with a high weight it's going to end up with a high probability.

- If we have this function, then we just need some way to update the weights to make them a little bit better.

- With such an approach, we can repeat this step a number of times, making the weights better and better until they are as good as we can make them.

- Searching for the best vector `W` is a way to search for the best function for recognising a digit.

- To be more specific, below are the steps required to turn this function into a machine learning classifier:
  - *Initialize* the weights. We initialize the weights to random values since we have a routine to improve these weights.
  - For each image, use the weights to predict whether it appears to be a three or a seven.
  - Based on these predictions, calculate how good the model is (its `loss`).
  - Calculate the `gradient`, which measures for each weight, how changing that weight would change the loss.
  - `Step` (change) all the weights based on the `gradient` calculation.
  - Go back to step one and repeat the process.
  - Stop training when you decide that the model is good enough, or you don't want to wait any longer.

- These 7 steps are the key to the training of all deep learning models.

- This technique is called `Gradient Descent`

- Before applying these steps to our image classification problem, let's illustrate what they look like in a simpler case.

- We'll define a simple function (the quadratic) and assume it's our loss function and x is our weighted parameter.

In [ ]:
def f(x): return x**2

In [ ]:
plot_function(f, 'x', 'x**2')

- The sequence of steps we described above starts by picking some random value for a parameter, and calculating the value of the loss

In [ ]:
plot_function(f, 'x', 'x**2')
plt.scatter(-1.5, f(-1.5), color='red');

- We now look to see what would happen if we increased or decreased our parameter by a little bit - the `adjustment`.

- This is simply the slope at a particular point.

- We can change our weight by a little in the direction of the slope, calculate our loss and adjustment again, and repeat this a few times.

- Eventually, we'll get to the lowest point on the curve.

- The basic idea goes back to Isaac Newton, who pointed out that we can optimize arbitrary functions this way.

- Regardless of how complicated our functions become, this basic approach of gradient descent will not significantly change.

- The only minor changes are some handy ways we can make it faster, by finding better steps.

## The gradient

- The one magic step is the bit where we calculate the *gradient*.

- We use calculus as a performance optimization; it allows us to more quickly calculate whether our loss will go up or down when we adjust our parameters up or down.

- In other words, the gradients will tell us how much we have to change each weight to make our model better.

- From highschool math, gradient is defined as "rise/run" i.e. the change in the value of the function divided by the change in the value of the parameter.

- The *derivative* of a function tells you how much a change in the parameters of a function will change its result.

- It is a function that calculates the change rather than the value.

- The derivative of the quadratic function at the value three tells us how rapidly the function changes at the value three.

- When we know how our function will change, then we know what to do to make it smaller.

- This is the key to machine learning: having a way to change the parameters of a function to make it smaller.

- Our function has lots of weights that we need to adjust, so when we calculate the derivative, we won't get back one number but lots of them - a gradient for every weight.

- You can calculate the derivative with respect to one weight and treat all the others as constant, then repeat that for each weight.

- This is how gradients are calculated for every weight.

- Luckily PyTorch is able to automatically compute the derivative of any function so you don't have to calculate any gradients yourself.

In [ ]:
# Pick a tensor value which we want gradients at
xt = tensor(3.).requires_grad_()

- `requires_grad_()` tells PyTorch that we want to calculate gradients with respect to that variable at that value.

- An underscore at the end of a method in PyTorch means that it is an `in-place operation`.

- An `in-place operation` directly changes the content of a given linear algebra, vector or tensor without making a copy.

- In our function above `requires_grad_` modifies `tensor(3.)` to tell PyTorch that we want to calculate gradients on it.

- We now calculate our function with that value.

In [ ]:
yt = f(xt)
yt

- Finally, we tell PyTorch to calculate the gradient for us

In [ ]:
yt.backward()

- `backward()` refers to `back propagation` which is the name given to the process of calculating the derivative of each layer.

- We can now view the gradients by checking the `grad` attribute of our tensor

In [ ]:
xt.grad

In [ ]:
xt = tensor([3.,4.,10.]).requires_grad_()
xt

In [ ]:
def f(x): return (x**2).sum()

yt = f(xt)
yt

- Our gradients are 2*x as expected

In [ ]:
yt.backward()
xt.grad

- The gradient only tells us the slope of our function, it doesn't actually tell us exactly how far to adjust the parameters.

- It gives us some idea of how far, if the slope is very large it suggests that we have more adjustments to do.

- Whereas if the slope is very small it may suggest that we are close to the optimal value.

## Stepping with a learning rate

- Deciding how to change our parameters based on the value of the gradients is an important part of the deep learning process.

- Nearly all approaches start with the basic idea of multiplying the gradient by some small number called the `learning rate (LR)`.

- People often select a learning rate just by trying a few numbers and finding which one results in the best model after training. The learning rate is often a number between 0.001 and 0.1, although it could be any number.

- Once you've picked your learning rate, you can adjust your parameters using this function:
  ```
  w -= gradient(w) * lr
  ```

- This is known as `stepping` your parameters using an `optimizer step`.

- If you pick a learning rate that's too low it can mean having to do a lot of steps.

- Picking a learning rate that's too high is even worse, it can result in the loss getting worse or it may bounce around rather than actually diverging.

## An end-to-end SGD example

- Let's look at an SGD example and see how finding a minimum can be used to train a model to fit data better.

- Imagine you were measuring the speed of a roller coaster as it went over the top of a hump. It would start fast, and then get slower as it went up the hill, and then would be slowest at the top, and it would speed up as it went down the hill. You want to build a model of how the space changes over time. If you're measuring the speed manually every second for 20 seconds, it might look something like this

In [ ]:
time = torch.arange(0,20).float(); time

In [ ]:
# torch.randn(20)*3 adds a random number to every observation since measuring things manually isn't precise
# The speed is calculated as the result of a quadratic function
speed = torch.randn(20)*3 + 0.75*(time - 9.5)**2 + 1
plt.scatter(time, speed)

- ```torch.randn(20)*3``` adds a bit of random noise since measuring things manually isn't precise.

- Using SGD we can try to find a function that matches our observations.

- We can't consider every possible function, so let's use a guess that it'll be quadratic, i.e. a function of the form `a*(time**2) + (b*time) + c`.

- We want to distinguish clearly between the functions input (the time we're measuring the coaster's speed) and its parameters (the values that define which quadratic we're trying).

- Let's collect the parameters in one argument and thus separate the input `t` and the parameter `params` in the function's signature.

In [ ]:
def f(t, params):
  a,b,c = params
  return a*(t**2) + (b*t) + c

- We've restricted the problem of finding the best imaginable function that fits the data to finding the best quadratic function.

- This greatly simplifies the problem, since every quadratic function is fully defined by the three parameters `a`, `b`, and `c`.

- To find the best quadratic function, we need to find the best values for `a`, `b`, and `c`.

- If we can solve the problem for the three parameters of the quadratic function, we'll be able to apply the same approach for other more complex functions with more parameters such as a neural net.

- We need to define what we mean by "best". We define this precisely by choosing a *loss function*, which will return a value based on a prediction and a target, where lower values of the function correspond to "better" predictions.

- For continuous data, it's common to use *mean squared error*

In [ ]:
def mse(preds, targets): return ((preds - targets)**2).mean()

- Let's work through our 7 step process:

**Step 1**:
- Initialize parameters to random values and tell PyTorch that we want to track their gradient using `requires_grad_`

In [ ]:
params = torch.randn(3).requires_grad_()

In [ ]:
orig_params = params.clone()

**Step 2:**
- Calculate the predictions

In [ ]:
preds = f(time, params)

- Let's create a little function to see how close our predictions are to our targets and take a look.

In [ ]:
def show_preds(preds, ax=None):
  if ax is None: ax = plt.subplots()[1]
  ax.scatter(time, speed)
  ax.scatter(time, to_np(preds), color='red')
  ax.set_ylim(-300, 100)

In [ ]:
show_preds(preds)

- This doesn't look very close -- our random parameters suggest that the roller coaster will end up going backwards since we have negative speeds

**Step 3:**
- Calculate the loss

In [ ]:
loss = mse(preds, speed)
loss

- Our goal is to improve this. To do that we'll need the gradients

**Step 4:**
- Calculate the gradients (an approximation of how the parameters need to change).

In [ ]:
loss.backward()
params.grad

- We pick a learning rate of $10^{-5}$

In [ ]:
params.grad * 1e-5

- We can use these gradients to improve our parameters

In [ ]:
params

**Step 5:**
- Step the weights (update the parameters based on the gradients we just calculated)

- We use the `.data` attribute in PyTorch to make sure that the gradient isn't calculated of the actual step we're doing.

- The gradient should only be calculate of the function `f`.

In [ ]:
lr = 1e-5
params.data -= lr * params.grad.data
params.grad = None  # remove the gradients previously calculated.

- Let's see if the loss improved

In [ ]:
preds = f(time, params)
mse(preds, speed)

- We notice that the loss has indeed reduced from the initial calculation.

- We also plot the function

In [ ]:
show_preds(preds)

- The plot has also improved.

- We need to repeat this a few times, so we'll create a function to apply one step.

In [ ]:
def apply_step(params, prn=True):
  preds = f(time, params)
  loss = mse(preds, speed)
  loss.backward()
  params.data -= lr * params.grad.data
  params.grad = None
  if prn: print(loss.item())
  return preds

**Step 6:**
- We repeat the process.

- By looping and performing many improvements, we hope to reach a good result.

In [ ]:
for i in range(10): apply_step(params)

In [ ]:
params = orig_params.detach().requires_grad_()

- The loss is going down just as we'd hoped.

- Looking only at the loss disguises the fact that each iteration represents an entirely different quadratic function being tried on the way to find the best possible quadratic function.

- We can see this process visually if, instead of printing out the loss, we plot the function at every step.

- Then we can see how the shape is approaching the best possible quadratic function for our data.

In [ ]:
_, axs = plt.subplots(1, 4, figsize=(12,3))
for ax in axs: show_preds(apply_step(params, False), ax)
plt.tight_layout()

**Step 7:**

- Stop.

- We decided to stop arbitrarily after 10 epochs.

- In practice, we watch the training and validation losses and our metrics decide when to stop.

# MNIST loss function

- We need gradients in order to improve our model using SGD, and in order to calculate gradients, we need some loss function that represents how good our model is.

- Gradients are a measure of how that loss function changes with small tweaks to the weights.

- In choosing a loss function, the obvious approach would be to use accuracy, which is our metric, as our loss function as well.

- In this case, we would calculate our prediction for each image, collect these values to calculate an overall accuracy, and then calculate the gradients of each weight with respect to the overall accuracy.

- The gradient of a function is how much the value of a function goes up or down divided by how much you changed the inputs.

- Accuracy only changes when a prediction changes from a 3 to a 7 or vice versa meaning that a small change in weights isn't likely to cause any prediction change.

- Therefore the problem with accuracy is that the gradient will be zero almost everywhere because a very small change in the value of a weight will often not change the accuracy at all.

- This makes accuracy a poor choice as a loss function because with the gradients as zero the model is unable to learn from that number.

- We need a loss function which, when our weights result in slightly better predictions, gives us a slightly better loss.

- We already have our `x`'s i.e. the images themselves.

- We'll concatenate them into a single tensor using `torch.cat()`, and also change them from a list of matrices (rank 3 tensor) to a list of vectors (rank 2 tensor) using the `view` method.

- `view` is a PyTorch method that changes the shape of a tensor without changing its contents.

- `-1` is a special parameter to `view` that means make this axis as big as necessary to fit all the data

In [ ]:
# view(-1, 28*28) -> put everything in as many rows of length 28*28 as is necessary
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

- We need a label for each. We'll use 1 for threes and 0 for sevens

In [ ]:
# unsqueeze(1) create a matrix with one column
train_y = tensor([1] * len(threes) + [0] * len(sevens)).unsqueeze(1)
train_x.shape, train_y.shape

- A dataset in PyTorch is required to return a tuple of (x, y) when indexed.

- Python provides a `zip` function which when combined with a list, provides a simple way to get this functionality.

In [ ]:
dset = list(zip(train_x, train_y))
x,y = dset[0]  # destructuring the tuple
x.shape, y

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1] * len(valid_3_tens) + [0] * len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))

- We initialize a random weight for every pixel

In [ ]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [ ]:
weights = init_params((28*28, 1))  # the 1 tells Pytorch that we want the results as a column

- The function `weights*pixels` won't be flexible enough.

- It's always equal to zero when the pixels are equal to zero i.e. its intercept is zero.

- The formula for a line is `y=w*x_b`; we still need the b

In [ ]:
bias = init_params(1)

In [ ]:
bias

- In neural networks, the `w` in the equation `y=w*x+b` is called the *weights* and `b` is called the *bias*.

- Together, the weights and bias make up the parameters.

- We can now calculate the prediction for one image.

In [ ]:
(train_x[0]*weights.T).sum() + bias

- We don't use a python for loop to calculate the prediction of each image.

- This is because for loops are very slow and don't run on the GPU.

- We need to represent as much of the computation in a model as possible using higher-level functions

- We can use *matrix multiplication* to calculate `w*x` for every row of a matrix.

- In Python, matrix multiplication is represented with the `@` operator

In [ ]:
def linear1(xb): return xb@weights + bias
preds = linear1(train_x)
preds

- The `batch@weights + bias` equation is one of the two fundamental equations of any neural network (the other one is the *activation function*).

- To decide if an output represents a 3 or a 7, we can just check if its greater than zero.

- Our accuracy for each item can be calculated with:

In [ ]:
corrects = (preds>0.0).float() == train_y  # broadcasting
corrects

In [ ]:
# .item() unwraps a tensor to create a normal python scalar
corrects.float().mean().item()

- Our randomly initialized model is right about half the time at predicting 3's from 7's

- Let's see what the change in accuracy is for a small change in one of the weights.

In [ ]:
weights[0] *= 1.0001

In [ ]:
preds = linear1(train_x)
((preds>0.0).float() == train_y).float().mean().item()

- We get back the exact same figure.

- We can see that we need gradients in order to improve our model using SGD, and in order to calculate gradients we need some *loss function* that represents how good our model is.

- Gradients are a measure of how that loss function changes with small tweaks to the weights.

- Accuracy as a loss function will not be useful in this case, due to the gradient being zero most of the time.

- We need a loss function which, when our weights result in slightly better predictions, gives us a better loss.

- The loss function receives not the images themselves, but the prediction from the model.

- One of the arguments to the loss function will be `predictions`, a vector (rank-1 tensor), indexed over the images, of values between 0 and 1, where each value is the prediction indicating how likely the component's image is a 3.

- The purpose of the loss function is to measure the difference between predicted values and true values (labels/targets).

- The loss will be smaller i.e. better when the predictions are closer to the targets.

- Another argument to the loss function will be the `targets`, a vector indexed over the images, with a value of 0 or 1 that tells us whether the image is actually a 3.

- For example, let's say we had three images of a 3, a 7 and a 3. Suppose our model predicted with high confidence the first was a 3, slight confidence the second was a 7 and fair confidence (incorrectly) that the last was a 7. This would mean our loss function would receive these values as inputs:

In [ ]:
trgts = tensor([1,0,1])
prds = tensor([0.9, 0.4, 0.2])

- Here's a first try at a loss function that measures the distance between predictions and targets:

In [ ]:
def mnist_loss(predictions, targets):
  """Return 1-predictions where targets==1 and predictions where targets==0"""
  return torch.where(targets==1, 1-predictions, predictions).mean()

- `torch.where(a, b, c)` is the equivalent of running the list comprehension `[b[i] if a[i] else c[i] for i in range(len(a))], except it works on tensors, at C/CUDA speeds.

- Our function measures how distant each prediction is from 1 if it should be 1, and how distant it is from 0 if it should be 0, and then takes the mean of all those distances.

In [ ]:
torch.where(trgts==1, 1-prds, prds)

In [ ]:
mnist_loss(prds, trgts)

- If we change the prediction for the one "false" target from 0.2 to 0.8, the loss will go down indicating that this is a better prediction

In [ ]:
mnist_loss(tensor([0.9, 0.4, 0.8]), trgts)

- `mnist_loss` as it is currently defined assumes that predictions are always between zero and one.

- We need to ensure that this is always the case.

- The `sigmoid function` always outputs a number between zero and one

## Sigmoid

- The `sigmoid` function is defined by:

In [ ]:
# torch.exp is the same as e
def sigmoid(x): return 1/(1+torch.exp(-x))

In [ ]:
math.e

In [ ]:
math.e**2 == torch.exp(tensor(2.))

- PyTorch already defines this for us.

- The sigmoid function is important in deep learning since we always want to ensure values are between zero and one.

In [ ]:
plot_function(torch.sigmoid, title='Sigmoid', min=-4, max=4)

- The sigmoid function takes in any input value, positive or negative and outputs a value between zero and one.

- Smaller numbers give an output closer to zero and larger numbers give an output closer to one.

- It never goes below zero and beyond one.

- It's also a smooth curve that only goes up, making it easier for SGD to find meaningful gradients.

- We update `mnist_loss` to apply `sigmoid` to the inputs

In [ ]:
def mnist_loss(predictions, targets):
  predictions = predictions.sigmoid()
  return torch.where(targets==1, 1-predictions, predictions).mean()

- We can now be confident that our loss function will work, even when the predictions are not between 0 and 1.

- Metrics, are the numbers we really care about i.e. accuracy.

- These are things printed out at the end of every epoch that tell us how our model is really doing.

- When judging the performance of a model it is important to focus on the metrics.

- The loss function needs to respond to small changes in confidence level.

- This means that sometimes the loss doesn't really reflect what we are trying to achieve, but is rather a compromise between our real goal, and a function that can be optimized using its gradient.

- The loss function is calculated for each item in our dataset, and at the end of an epoch these are all averaged, and the overall mean is reported for the epoch.

- In summary, the key difference between the loss and the metric is that the metric is to drive human understanding and the loss is to drive automated learning.

### SGD and mini-batches

- Now that we have a loss function that is suitable to drive SGD, we can now consider the `optimization step`.

- The optimization step is where we step (change or update) the weights based on the gradient.

- To do this we need to calculate the loss of one or more data items.

- We calculate the loss of a few data items at a time, called a `mini-batch`.

- The number of items in a mini-batch is called a `batch size`.

- We take mini-batches because:
  - Calculating the loss for the whole dataset takes a very long time.
  - Calculating the loss for each individual item does not use much information and results in a very imprecise and unstable gradient.

- A larger batch size means that you will get a more accurate and stable estimate of your dataset's gradient on the loss function, but it takes longer and you will get less mini-batches per epoch.

- Mini-batches are ideal since we train on a GPU and GPUs only perform well if they have lots of work to do at a time.

- From data augmentation, we get better generalization if we vary things during training.

- A simple and effective thing we can do is to randomly shuffle our dataset on every epoch before we create mini-batches.

- PyTorch and fastai provide a `DataLoader` that does the shuffling and mini batch collation for you.

In [ ]:
coll = range(15)
dl = DataLoader(coll, batch_size=5, shuffle=True)
list(dl)

- `Dataset` - A PyTorch collection that contains tuples of independent and dependent variables

- For training a model, we just don't want any collection but a PyTorch collection i.e. a *dataset*

In [ ]:
ds = L(enumerate(string.ascii_lowercase))
ds

- When we pass a Dataset to a Dataloader, we get back many batches that are themselves tuples of tensors representing batches of independent and dependent variables.

In [ ]:
dl = DataLoader(ds, batch_size=6, shuffle=True)
list(dl)

### Putting it all together

- We can now write our first training loop for a model using SGD.

- In code, our process will be implemented something like this for every epoch:
```python
    for x,y in dl:
        pred = model(x)
        loss = loss_func(pred, y)
        loss.backward()
        parameter -= parameter.grad - lr
```

- First, let's re-initialize our parameters:

In [ ]:
weights = init_params((28*28,1))
bias = init_params(1)

- A `DataLoader` can be created from a `Dataset`

In [ ]:
dl = DataLoader(dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape

- We'll do the same for the validation set.

In [ ]:
valid_dl = DataLoader(valid_dset, batch_size=256)

- Let's create a mini-batch of size 4 for testing

In [ ]:
batch = train_x[:4]
batch.shape

In [ ]:
preds = linear1(batch)
preds

In [ ]:
loss = mnist_loss(preds, train_y[:4])
loss

- Now we calculate the gradients:

In [ ]:
loss.backward()
weights.grad.shape, weights.grad.mean(), bias.grad

- Let's put all that in a function

In [ ]:
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss.backward()


In [ ]:
calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(), bias.grad

In [ ]:
calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(), bias.grad

- When we call the function a second time, notice that the gradients have changed. This is very concerning.

- `loss.backward` not only calculates the gradient, it also adds the gradients of `loss` to any gradients that have been stored.

- To prevent this from happening we have to set the current gradients to zero first.

In [ ]:
weights.grad[541]

In [ ]:
# change the weights.grad tensor to be a tensor containing zeros
# the _ just means do it in place
weights.grad.zero_()
bias.grad.zero_();

- The remaining step will be to update the weights and bias based on the gradient and learning rate.

- We have to be careful and tell PyTorch not to take the gradient of this step too, otherwise it gets very confusing when we try to compute the derivative at the next batch.

- We do this by assigning to the `data` attribute of a tensor

In [ ]:
def train_epoch(model, lr, params):
  """A basic standard SGD loop."""
  for xb, yb in dl:
    calc_grad(xb, yb, model)
    for p in params:
      p.data -= p.grad*lr
      p.grad.zero_()

> The difference between `gradient descent` and `stochastic gradient descent` is that gradient descent calculates the gradient on the entire dataset while stochastic gradient descent loops through a mini-batch and calculates the gradient on that

- We also want to know how we're doing by looking at the accuracy of the validation set.

- To decide if an input represents a 3 or a 7, we can just check whether it's greater than zero.

- The accuracy of each item can be calculated (using broadcasting) with:

In [ ]:
(preds>0.0).float() == train_y[:4]

- The above gives us this function to calculate our validation accuracy:

In [ ]:
def batch_accuracy(xb, yb):
  preds = xb.sigmoid()
  correct = (preds>0.5) == yb
  return correct.float().mean()

- You'll notice that we're working with the sigmoid of the predictions.

- To measure the accuracy we use `(preds>0.5)`, this is because on the sigmoid curve 0 corresponds to 0.5

In [ ]:
batch_accuracy(linear1(batch), train_y[:4])

In [ ]:
def validate_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
  return round(torch.stack(accs).mean().item(), 4)

In [ ]:
validate_epoch(linear1)

- Let's train for one epoch and see if the accuracy improves.

In [ ]:
lr = 1.
params = weights,bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

In [ ]:
for i in range(20):
  train_epoch(linear1, lr, params)
  print(validate_epoch(linear1), end=' ')

- We're already at about the same accuracy as our "pixel similarity" approach, and we've created a general purpose foundation we can build on.

- The next step will be to create an object that will handle the SGD step for us. In PyTorch, it's called an *optimizer*.

## Creating an optimizer

- PyTorch provides some useful classes to make implementation easier.

- A "module" is an object of a class that inherits from the PyTorch `nn.module` class.

- Objects of this class behave like a standart Python function, in that you can call it using parentheses, and it will return activations of a model.

- First, we'll replace our `linear1()` function with PyTorch's `nn.Linear` module.

- `nn.Linear` does the same thing as our `init_params` and `linear1` together.

- It contains both the weights and bias in a single class

In [ ]:
linear_model = nn.Linear(28*28, 1)

- Every PyTorch module knows what parameters it has that can be trained; they are available through the `parameters` method.

In [ ]:
w,b = linear_model.parameters()
w.shape, b.shape

- We can use this information to create an optimizer

In [ ]:
class BasicOptim:
  def __init__(self, params, lr): self.params, self.lr = list(params), lr

  def step(self, *args, **kwargs):
    for p in self.params: p.data -= p.grad.data * self.lr

  def zero_grad(self, *args, **kwargs):
    for p in self.params: p.grad = None

- We can create our optimizer by passing in the model's parameters:

In [ ]:
opt = BasicOptim(linear_model.parameters(),lr)

- Our training loop can now be simplified to:

In [ ]:
def train_epoch(model):
  for xb, yb in dl:
    calc_grad(xb, yb, model)
    opt.step()
    opt.zero_grad()

- Our validation function doesn't need to change

In [ ]:
validate_epoch(linear_model)

- Putting our training loop into a function:

In [ ]:
def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end=' ')

In [ ]:
train_model(linear_model, 20)

- fastai provides the `SGD` class which, by default, does the same thing as our `BasicOptim`

In [ ]:
linear_model = nn.Linear(28*28, 1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

- fastai also provides a `Learner.fit` that can be used instead of `train_model`.

- To create a `Learner` we first need to create `DataLoaders` by passing in our training and validation `DataLoader`.

- `DataLoaders` stores away a `DataLoader` as a `.train` and a `.valid`.

- With `DataLoaders` we now have a sigle object that knows all the data we have.

- It can make sure that the training `DataLoader` is shuffled and the validation `DataLoader` isn't shuffled so that everything works properly.

In [ ]:
dls = DataLoaders(dl, valid_dl)

- To create a `Learner` class without using an application (such as `cnn_learner`), we need to pass in: the `DataLoaders`, the model, the optimization function, the loss function, and optionally any metrics to print

In [ ]:
learn = Learner(dls, nn.Linear(28*28, 1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

- We can now call `fit`

In [ ]:
learn.fit(10, lr=lr)

- With these classes, we can now replace a linear function with a neural network

## Adding a non-linearity

- A linear classifier is very constrained in terms of what it can do.

- To make it a bit more complex (and able to handle more tasks), we need to add a non-linearity between two linear classifiers, and this is what gives us a neural network.

- Below is the entire definition of a basic neural network:

In [ ]:
def simple_net(xb):
  """res.max(tensor(0.0)) takes any negative numbers and turns them into zeros.
  The first and third lines are known as linear layers.
  The second line is known as a nonlinearity or activation function.
  """
  res = xb@w1 + b1
  res = res.max(tensor(0.0))  # recified linear unit (ReLU)
  res = res@w2 + b2
  return res

- Here, `w1` and `w2` are weight tensors and `b1` and `b2` are bias tensors; i.e. parameters that are initially randomly initialized.

- Our `simple_net` function is an example of *function composition*.

- `Function composition` - take the result of a function and pass it to a new one.

- Every neural network is basically doing function composition of linear layers and activation functions.

In [ ]:
w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

- `w1` has 30 output activations (meaning that `w2` must have 30 input activations so that they match).

- This means that the first layer can construct 30 different features, each representing some different mix of pixels.

- You can change the `30` to anything you like, to make the model more or less complex.

- The function `res.max(tensor(0.0))` is called a *rectified linear unit (ReLU)*.

- It replaces every negative number with zero.

- It is also available in PyTorch as `F.relu`:

In [ ]:
plot_function(F.relu)

- According to the `Universal Approximation Theorem` if the size of the weights and bias matrices are big enough and you can find the right parameters, then our `simple_net` function can solve any computable problem to an arbitrarily high level of accuracy (including the function of how to recognize 3s and 7s.

- This is only possible because of the `max` function that adds a non-linearity to our `simple_net` function.

- Mathematically, the composition of two linear functions is a linear function. Without non-linear functions, we can stack as many linear classifiers on top of each other and it will just be the same as one linear classifier.

>> A series of any number of linear layers in a row can be replaced with a single linear layer with a different set of parameters.

- We can replace our `simple_net` code with something a bit simpler by taking advantage of PyTorch:

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30,1)
)

- `nn.Sequential` creates a module which will call each of the listed layers or functions in turn. It is an easier way to do function composition.

- `F.relu` is a function, not a PyTorch module.

- `nn.ReLU` is a PyTorch module that does exactly the same thing.

- When using `nn.Sequential` PyTorch requires us to use the module version and since modules are classes, we have to instantiate them i.e. `nn.ReLU()`.

In [ ]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(40, 0.1)

> For deeper models, you may want to use a lower learning rate and a few more epochs.

- The training process is recorded in `learn.recorder`, with the table of output stored in the `values` attribute.

- We can plot the accuracy over training as:

In [ ]:
# itemgot(2) get the second item from every row (batch_accuracy) for plotting
plt.plot(L(learn.recorder.values).itemgot(2))

- ...and view the final accuracy:

In [ ]:
learn.recorder.values[-1][2]

- At this point we have:
  - A function that can solve any problem to any level of accuracy (the neural network) given the correct set of parameters.
  - A way to find the best set of parameters for any function (stochastic gradient descent).

- Below we see a way to look into what the model is doing.

- To understand the features that make up the first layer, we can plot them as seen below. For layers other than the first though we have to be a lot more sophisticated.

In [ ]:
m = learn.model
m

In [ ]:
m[0].parameters()

In [ ]:
w,b = m[0].parameters()
w.shape, b.shape

In [ ]:
show_image(w[0].view(28,28))

- In practice, we can add as many linear layers as we want as long as we add a nonlinearity between each pair of linear layers (making the model deeper).

- However, the deeper the model gets, the harder it is to optimise the parameters in practice.

- **Why use deeper models?**
> Performance. With a deeper model, we do not need to use as many parameters; we can use smaller matrices, with more layers, and get better results than we would with larger matrices and fewer layers. We can therefore train quicker and our model takes up less memory.

- Below we see what happens when we train an 18 layer model using the same approach:

In [ ]:
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.060807,0.011237,0.998037,02:33


- It gives nearly 100% accuracy, a big difference compared to our `simple_net` function.

### Summary of key concepts related to SGD.

> Activations - numbers that are calculated.

> Parameters - numbers that are randomly initialized, and optimised (the numbers we're learning).

> ReLU - A function that returns zero for negative numbers and doesn't change for positive numbers.

> Mini-batch - A small batch of inputs and labels gathered together in two arrays. A gradient descent step is updated on this batch (rather than a whole epoch).

>Forward pass - Applying the model to some input and computing the predictions.

> Loss - A value that represents how well (or badly) our model is doing.

> Gradient - The derivative of the loss with respect  to some parameter of the model.

> Backward pass - Computing the gradients of the loss with respect to all model parameters.

> Gradient descent - Taking a step in the directions opposite to the gradients to make the model parameters a little bit better.

> Learning rate - The size of the step we make when applying SGD to update the parameters of the model.